In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction\\research'

In [3]:
os.chdir('../')
%pwd

'f:\\Files\\DS&ML\\Tele-Com-Customer-Churn-Prediction'

In [4]:
from mlproject.constants import *

In [5]:
from dataclasses import dataclass
from pathlib import Path
import os
import pandas as pd
import numpy as np
import joblib
import json
import mlflow
import dagshub
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import logging

In [6]:
from dataclasses import dataclass
import pandas as pd
import joblib
import json
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json
from mlproject import logger

In [7]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    metric_file_path: Path
    preprocessor_path: Path
    test_raw_data: Path  
    target_column: str
    all_params: dict

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LGBMClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            metric_file_path=config.metric_file_path,
            preprocessor_path=config.preprocessor_path,
            test_raw_data=config.test_raw_data,  # Using the correct attribute name from config
            target_column=schema.name,
            all_params=params
        )
        return model_evaluation_config

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
        # Initialize MLflow tracking
        os.environ['MLFLOW_TRACKING_USERNAME'] = "JavithNaseem-J"
        os.environ['MLFLOW_TRACKING_PASSWORD'] = "39af2ec9240d8439ca7a568d37e4c8566f0e4507"
        
        dagshub.init(repo_owner="JavithNaseem-J", repo_name="Telecom-Customer-Churn-Prediction")
        mlflow.set_tracking_uri("https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow")
        mlflow.set_experiment("Telecom-Customer-Churn-Prediction")

    def evaluate(self):
        try:
            # Validate file paths using Path objects for cross-platform compatibility
            if not Path(self.config.test_raw_data).exists():
                raise FileNotFoundError(f"Test data file not found at {self.config.test_raw_data}")
            if not Path(self.config.preprocessor_path).exists():
                raise FileNotFoundError(f"Preprocessor file not found at {self.config.preprocessor_path}")
            if not Path(self.config.model_path).exists():
                raise FileNotFoundError(f"Model file not found at {self.config.model_path}")

            with mlflow.start_run():
                # Set tags for the run
                mlflow.set_tag("model_type", "CatBoostClassifier")
                mlflow.set_tag("evaluation_stage", "testing")

                # Load preprocessor and model
                logger.info("Loading preprocessor and model...")
                preprocessor = joblib.load(self.config.preprocessor_path)
                model = joblib.load(self.config.model_path)

                # Log model parameters
                mlflow.log_params(self.config.all_params)

                # Load test data
                logger.info(f"Loading test data from {self.config.test_raw_data}...")
                test_data = pd.read_csv(self.config.test_raw_data)
                
                # Use the original column names instead of converting to lowercase
                target_column = self.config.target_column  # Keep original case

                # Extract target column
                if target_column not in test_data.columns:
                    raise KeyError(f"Target column '{target_column}' not found in test data. Available columns: {list(test_data.columns)}")

                test_y = test_data[target_column]
                test_x = test_data.drop(columns=[target_column])

                logger.info(f"Test data shape: X={test_x.shape}, y={test_y.shape}")

                # Preprocess test features
                logger.info("Preprocessing test features...")
                test_x_transformed = preprocessor.transform(test_x)

                # Make predictions
                logger.info("Making predictions on the test data...")
                predictions = model.predict(test_x_transformed)

                # Calculate and log metrics
                logger.info("Evaluating model performance...")
                
                # Handle potential warnings for classification metrics
                try:
                    precision = precision_score(test_y, predictions, average="weighted")
                    recall = recall_score(test_y, predictions, average="weighted")
                    f1 = f1_score(test_y, predictions, average="weighted")
                except Exception as metric_error:
                    logger.warning(f"Error calculating some metrics: {str(metric_error)}. Using zero-division=0.")
                    precision = precision_score(test_y, predictions, average="weighted", zero_division=0)
                    recall = recall_score(test_y, predictions, average="weighted", zero_division=0)
                    f1 = f1_score(test_y, predictions, average="weighted", zero_division=0)
                
                metrics = {
                    "accuracy": accuracy_score(test_y, predictions),
                    "precision": precision,
                    "recall": recall,
                    "f1": f1
                }

                # Log metrics to MLflow
                mlflow.log_metrics(metrics)

                # Log model with signature
                signature = mlflow.models.infer_signature(
                    test_x_transformed, predictions
                )
                mlflow.sklearn.log_model(
                    model,
                    "TelecomCustomerChurnModel",
                    signature=signature,
                    registered_model_name="TelecomCustomerChurnModel"
                )

                logger.info(f"Model Evaluation Metrics:\naccuracy: {metrics['accuracy']}\n"
                          f"precision: {metrics['precision']}\nrecall: {metrics['recall']}\n"
                          f"f1: {metrics['f1']}")

                # Save metrics locally using Path for cross-platform compatibility
                metrics_file = Path(self.config.root_dir) / "metrics.json"
                with open(metrics_file, "w") as f:
                    json.dump(metrics, f, indent=4)
                    
                logger.info(f"Evaluation metrics saved at {metrics_file}")

                return metrics

        except Exception as e:
            logger.error(f"Error in model evaluation: {str(e)}")
            raise e

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration or data: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-03-04 11:43:30,554: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-04 11:43:30,559: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-04 11:43:30,569: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-04 11:43:30,574: INFO: common: created directory at: artifacts]
[2025-03-04 11:43:30,582: INFO: common: created directory at: artifacts/model_evaluation]
[2025-03-04 11:43:31,167: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as JavithNaseem-J

[2025-03-04 11:43:31,179: INFO: helpers: Accessing as JavithNaseem-J]
[2025-03-04 11:43:31,791: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/JavithNaseem-J/Telecom-Customer-Churn-Prediction "HTTP/1.1 200 OK"]
[2025-03-04 11:43:32,356: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "JavithNaseem-J/Telecom-Customer-Churn-Prediction"

[2025-03-04 11:43:32,361: INFO: helpers: Initialized MLflow to track repo "JavithNaseem-J/Telecom-Customer-Churn-Prediction"]


...

[2025-03-04 11:43:32,365: INFO: helpers: Repository JavithNaseem-J/Telecom-Customer-Churn-Prediction initialized!]
[2025-03-04 11:43:34,159: INFO: 2214911065: Loading preprocessor and model...]
[2025-03-04 11:43:35,024: INFO: 2214911065: Loading test data from artifacts/data_transformation/test.csv...]
[2025-03-04 11:43:35,053: INFO: 2214911065: Test data shape: X=(2587, 19), y=(2587,)]
[2025-03-04 11:43:35,055: INFO: 2214911065: Preprocessing test features...]
[2025-03-04 11:43:35,062: INFO: 2214911065: Making predictions on the test data...]
[2025-03-04 11:43:35,224: INFO: 2214911065: Evaluating model performance...]


f:\ProgramFiles\anaconda3\envs\mlproject\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
Successfully registered model 'TelecomCustomerChurnModel'.
2025/03/04 11:43:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: TelecomCustomerChurnModel, version 1


[2025-03-04 11:43:50,932: INFO: 2214911065: Model Evaluation Metrics:
accuracy: 0.847313490529571
precision: 0.8474327707126296
recall: 0.847313490529571
f1: 0.8473061440986339]
[2025-03-04 11:43:50,934: INFO: 2214911065: Evaluation metrics saved at artifacts\model_evaluation\metrics.json]


Created version '1' of model 'TelecomCustomerChurnModel'.


🏃 View run awesome-lynx-1000 at: https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow/#/experiments/1/runs/10c2e183442b4ba49e0e05d9a3adeb38
🧪 View experiment at: https://dagshub.com/JavithNaseem-J/Telecom-Customer-Churn-Prediction.mlflow/#/experiments/1
